In [1]:
from crewai import Crew
from dotenv import load_dotenv

from agents import linkedin_scraper_agent, web_researcher_agent, doppelganger_agent
from tasks import scrape_linkedin_task, web_research_task, create_linkedin_post_task

load_dotenv()


crew = Crew(
    agents=[
        linkedin_scraper_agent,
        web_researcher_agent,
        doppelganger_agent
    ],
    tasks=[
        scrape_linkedin_task,
        web_research_task,
        create_linkedin_post_task
    ]
)

result = crew.kickoff()


print("Here is the result: ")
print(result)


/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot 

Worker LLM (ollama/hf.co/unsloth/Mistral-Small-3.1-24B-Instruct-2503-GGUF:Q5_K_M) initialized.
# Agent: LinkedIn Post Scraper
## Task: Scrape a LinkedIn profile to get some relevant posts


# Agent: LinkedIn Post Scraper
## Using tool: ScrapeLinkedinPosts
## Tool Input: 
"{}"
## Tool Output: 
['Climbing the AI Agents Staircase – One Step Closer to the Future!Every day, I get more fired up about what’s possible with Agentic AI! From simple LLMs to fully autonomous systems capable of planning, decision-making, and even self-learning—this is the frontier of innovation, and we’re right in the thick of it.This awesome visual below breaks it all down—from the basics to the advanced, showing how we move from connecting APIs and memory management… all the way up to agents that can think, act, collaborate, and improve themselves.I just shared a new blog post where I dive into one of the most exciting parts of the journey:Designing autonomous planning and decision-making AI solutions.Check it ou

In [1]:
!Ollama list


NAME                                                             ID              SIZE      MODIFIED     
hf.co/unsloth/Mistral-Small-3.1-24B-Instruct-2503-GGUF:Q5_K_M    73cf47fe711c    16 GB     11 days ago     
exaone-deep:latest                                               106afe416a9e    4.8 GB    2 weeks ago     
hf.co/bartowski/open-r1_OlympicCoder-32B-GGUF:Q4_K_S             e849518c3f59    18 GB     2 weeks ago     
llama3.2:3b-beeai                                                c3bc94496dab    2.0 GB    2 weeks ago     
phi4-mini:3.8b-fp16                                              d40845b9e3a0    7.7 GB    2 weeks ago     
gemma3:27b                                                       30ddded7fba6    17 GB     3 weeks ago     
gemma3:12b                                                       6fd036cefda5    8.1 GB    3 weeks ago     
qwq:latest                                                       cc1091b0e276    19 GB     3 weeks ago     
hf.co/OuteAI/OuteTTS-0.3-500M-G

In [2]:
from crewai import Agent, Task, Crew, LLM
from tools import scrape_linkedin_posts_tool
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from tools.utils import load_yaml_file
from dotenv import load_dotenv
from textwrap import dedent
import os

load_dotenv("creds.env")

WORKER_MODEL = "ollama/hf.co/unsloth/Mistral-Small-3.1-24B-Instruct-2503-GGUF:Q5_K_M"
OLLAMA_BASE_URL = "http://localhost:11434"

ollama_llm = LLM(model=WORKER_MODEL, api_base=OLLAMA_BASE_URL, timeout=1200, temperature=0.1)

# Tools
tool_mapping = {
    "scrape_linkedin_posts_tool": scrape_linkedin_posts_tool,
    "scrape_website_tool": ScrapeWebsiteTool(),
    "search_tool": SerperDevTool()
}

# Load YAML
agent_defs = load_yaml_file("config/agents.yaml")
task_defs = load_yaml_file("config/tasks.yaml")

# Topic
topic = input("Enter your topic (e.g., 'Llama 3 vs Llama 2'): ")

# Create Agents
agents = {}
for name, config in agent_defs.items():
    agents[name] = Agent(
        role=config["role"],
        goal=config["goal"],
        backstory=dedent(config["backstory"]),
        tools=[tool_mapping[t] for t in config.get("assigned_tool_names", [])],
        verbose=True,
        allow_delegation=False,
        llm=ollama_llm
    )

# Create Tasks
tasks = {}
for name, config in task_defs.items():
    task = Task(
        description=dedent(config["description"]).replace("{{topic}}", topic),
        expected_output=dedent(config["expected_output"]).replace("{{topic}}", topic),
        agent=agents[config["agent"]],
    )
    tasks[name] = task

# Set Context
for name, config in task_defs.items():
    task = tasks[name]
    context_names = config.get("context_task_names", [])
    if context_names:
        task.context = [tasks[n] for n in context_names]

# Run Crew
crew = Crew(
    agents=list(agents.values()),
    tasks=list(tasks.values())
)

result = crew.kickoff()
print("Here is the result: ")
print(result)


Enter your topic (e.g., 'Llama 3 vs Llama 2'):  mcp server and its influnce in Agentic AI


# Agent: LinkedIn Post Scraper
## Task: Scrape a LinkedIn profile to get relevant posts.


# Agent: LinkedIn Post Scraper
## Using tool: ScrapeLinkedinPosts
## Tool Input: 
"{}"
## Tool Output: 
['Climbing the AI Agents Staircase – One Step Closer to the Future!Every day, I get more fired up about what’s possible with Agentic AI! From simple LLMs to fully autonomous systems capable of planning, decision-making, and even self-learning—this is the frontier of innovation, and we’re right in the thick of it.This awesome visual below breaks it all down—from the basics to the advanced, showing how we move from connecting APIs and memory management… all the way up to agents that can think, act, collaborate, and improve themselves.I just shared a new blog post where I dive into one of the most exciting parts of the journey:Designing autonomous planning and decision-making AI solutions.Check it out here: 🧠 Think, Plan, Solve: A Smarter Way to Tackle Problems with Multi-Agent Planning https://ln

In [4]:
from dotenv import load_dotenv

from tools import scrape_linkedin_posts_tool
from tools import scrape_linkedin_posts_fn

load_dotenv("creds.env")


print(scrape_linkedin_posts_fn())

['Climbing the AI Agents Staircase – One Step Closer to the Future!Every day, I get more fired up about what’s possible with Agentic AI! From simple LLMs to fully autonomous systems capable of planning, decision-making, and even self-learning—this is the frontier of innovation, and we’re right in the thick of it.This awesome visual below breaks it all down—from the basics to the advanced, showing how we move from connecting APIs and memory management… all the way up to agents that can think, act, collaborate, and improve themselves.I just shared a new blog post where I dive into one of the most exciting parts of the journey:Designing autonomous planning and decision-making AI solutions.Check it out here: 🧠 Think, Plan, Solve: A Smarter Way to Tackle Problems with Multi-Agent Planning https://lnkd.in/g_ne9teQLet’s build smarter agents, stronger teams, and shape what’s next.If you’re working with agentic systems, I’d love to hear what excites you most!hashtag#AI hashtag#AgenticAI hashtag

In [2]:
# !pip install selenium

  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 13.1 MB/s eta 0:00:00a 0:00:01
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)


In [ ]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.52:8501

/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function ca